# Demo 1: Traditional Approach with System Prompts

This notebook demonstrates the traditional approach to context engineering where we send a lengthy system prompt with **every** API request.

**Key Points:**
- System prompt sent with EVERY request
- High token overhead
- Consistent but expensive

## Setup

In [1]:
import os
import sys
from pathlib import Path

# Add utils to path
sys.path.append(str(Path(".").resolve().parent))

from utils.helpers import (
    load_system_prompt,
    count_tokens,
    get_sample_queries,
    format_comparison,
    calculate_cost
)

## Load the Expert System Prompt

This is a 551-token system prompt that defines RAG/Vector DB expertise.

In [2]:
system_prompt = load_system_prompt()
system_token_count = count_tokens(system_prompt)

print(f"System prompt tokens: ~{system_token_count}")
print(f"\n--- System Prompt Preview ---\n")
print(system_prompt[:500] + "...")

System prompt tokens: ~551

--- System Prompt Preview ---

You are an expert AI Solutions Architect with over 10 years of experience specializing in vector databases, retrieval-augmented generation (RAG) systems, and production AI deployments.

Your core expertise includes:

**Vector Databases & Search:**
- Deep knowledge of Weaviate, Pinecone, Qdrant, and Milvus architectures
- Expertise in embedding models (OpenAI, Cohere, sentence-transformers)
- Advanced understanding of ANN algorithms (HNSW, IVF, Product Quantization)
- Hybrid search strategies com...


## Mock OpenAI Client

For demonstration purposes. In production, use the actual OpenAI client.

In [3]:
class MockOpenAIClient:
    """Mock client for demonstration purposes"""
    def chat_completions_create(self, messages, model="gpt-4", **kwargs):
        user_query = messages[-1]["content"]
        return MockResponse(
            f"Based on my expertise in RAG systems and vector databases, "
            f"here's how I would approach '{user_query[:50]}...': "
            f"[Detailed expert response simulated here.]"
        )

class MockResponse:
    def __init__(self, content):
        self.choices = [type('obj', (object,), {'message': type('obj', (object,), {'content': content})()})()]

client = MockOpenAIClient()
print("Client initialized")

Client initialized


## Traditional Query Function

Notice how we include the full system prompt with **every single request**.

In [4]:
def traditional_query(client, system_prompt: str, user_query: str):
    """
    Execute a query using the traditional approach with system prompt
    """
    messages = [
        {"role": "system", "content": system_prompt},  # <-- Sent EVERY time!
        {"role": "user", "content": user_query}
    ]
    
    response = client.chat_completions_create(messages=messages, model="gpt-4")
    system_tokens = count_tokens(system_prompt)
    
    return response.choices[0].message.content, system_tokens

## Run Sample Queries

Let's run a few queries and see the token overhead.

In [5]:
queries = get_sample_queries()[:3]

print(f"Running {len(queries)} sample queries...\n")
print("=" * 80)

total_system_tokens = 0

for i, query in enumerate(queries, 1):
    response, system_tokens = traditional_query(client, system_prompt, query)
    total_system_tokens += system_tokens
    
    print(f"\nQuery {i}: {query[:60]}...")
    print(f"System tokens: {system_tokens}")
    print(f"Response: {response[:100]}...")
    print("-" * 80)

print(f"\nTotal system prompt tokens across {len(queries)} queries: {total_system_tokens}")

Running 3 sample queries...


Query 1: What's the best chunking strategy for long technical documen...
System tokens: 551
Response: Based on my expertise in RAG systems and vector databases, here's how I would approach 'What's the b...
--------------------------------------------------------------------------------

Query 2: How do I optimize Weaviate for 100M+ vectors?...
System tokens: 551
Response: Based on my expertise in RAG systems and vector databases, here's how I would approach 'How do I opt...
--------------------------------------------------------------------------------

Query 3: Should I use hybrid search or pure vector search for my use ...
System tokens: 551
Response: Based on my expertise in RAG systems and vector databases, here's how I would approach 'Should I use...
--------------------------------------------------------------------------------

Total system prompt tokens across 3 queries: 1653


## The Problem: Token Overhead at Scale

Let's visualize what this looks like at scale.

In [6]:
avg_user_tokens = 50
total_tokens_per_request = system_token_count + avg_user_tokens

print("TOKEN BREAKDOWN PER REQUEST")
print("=" * 50)
print(f"System Prompt:  {system_token_count:>6} tokens")
print(f"User Query:     {avg_user_tokens:>6} tokens (avg)")
print(f"-" * 50)
print(f"Total:          {total_tokens_per_request:>6} tokens")
print(f"\nSystem prompt is {(system_token_count/total_tokens_per_request)*100:.1f}% of every request!")

TOKEN BREAKDOWN PER REQUEST
System Prompt:     551 tokens
User Query:         50 tokens (avg)
--------------------------------------------------
Total:             601 tokens

System prompt is 91.7% of every request!


## Cost Analysis at Scale

In [7]:
scenarios = [
    ("10K requests/day", 10_000 * 30),
    ("100K requests/day", 100_000 * 30),
    ("1M requests/day", 1_000_000 * 30),
]

print("MONTHLY COST ANALYSIS (Traditional Approach)")
print("=" * 60)
print(f"{'Scenario':<25} {'Monthly Requests':>15} {'Monthly Cost':>15}")
print("-" * 60)

for name, monthly_requests in scenarios:
    total_tokens = total_tokens_per_request * monthly_requests
    cost = calculate_cost(total_tokens)
    print(f"{name:<25} {monthly_requests:>15,} ${cost:>14,.2f}")

print("\nThese costs are JUST for input tokens (system + user prompt)!")

MONTHLY COST ANALYSIS (Traditional Approach)
Scenario                  Monthly Requests    Monthly Cost
------------------------------------------------------------
10K requests/day                  300,000 $      5,409.00
100K requests/day               3,000,000 $     54,090.00
1M requests/day                30,000,000 $    540,900.00

These costs are JUST for input tokens (system + user prompt)!


## Comparison Preview

What if we could eliminate that system prompt overhead entirely?

In [8]:
metrics = format_comparison(
    traditional_tokens=total_tokens_per_request,
    baked_tokens=avg_user_tokens,  # Just the user query!
    num_requests=1_000_000
)

print("TRADITIONAL vs BAKED MODEL COMPARISON (1M requests)")
print("=" * 70)
print(f"{'Metric':<35} {'Traditional':>15} {'Baked':>15}")
print("-" * 70)
print(f"{'Tokens per request':<35} {metrics['traditional']['tokens_per_request']:>15} {metrics['baked']['tokens_per_request']:>15}")
print(f"{'Monthly cost (1M req)':<35} ${metrics['traditional']['cost']:>14,.2f} ${metrics['baked']['cost']:>14,.2f}")
print("-" * 70)
print(f"{'Token savings':<35} {metrics['savings']['tokens_percent']:>14.1f}%")
print(f"{'Monthly savings':<35} ${metrics['savings']['total_cost']:>14,.2f}")
print(f"{'Annual savings':<35} ${metrics['savings']['annual_cost_1m_requests']:>14,.2f}")

TRADITIONAL vs BAKED MODEL COMPARISON (1M requests)
Metric                                  Traditional           Baked
----------------------------------------------------------------------
Tokens per request                              601              50
Monthly cost (1M req)               $     18,030.00 $      1,500.00
----------------------------------------------------------------------
Token savings                                 91.7%
Monthly savings                     $     16,530.00
Annual savings                      $    198,360.00


## Key Insights

- System prompt adds **~550 tokens** to EVERY request
- At scale, this becomes a **significant cost driver**
- Longer prompts = **higher latency**
- Must maintain consistency across all deployments

## Next Step

Run **Demo 2** to see how Bread AI eliminates this overhead through "prompt baking"!